# Часть 1 Бустинг (5 баллов)

В этой части будем предсказывать зарплату data scientist-ов в зависимости  от ряда факторов с помощью градиентного бустинга.

В датасете есть следующие признаки:



* work_year: The number of years of work experience in the field of data science.

* experience_level: The level of experience, such as Junior, Senior, or Lead.

* employment_type: The type of employment, such as Full-time or Contract.

* job_title: The specific job title or role, such as Data Analyst or Data Scientist.

* salary: The salary amount for the given job.

* salary_currency: The currency in which the salary is denoted.

* salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

* employee_residence: The country or region where the employee resides.

* remote_ratio: The percentage of remote work offered in the job.

* company_location: The location of the company or organization.

* company_size: The company's size is categorized as Small, Medium, or Large.

In [1]:
import pandas as pd
!wget https://github.com/hse-ds/iad-intro-ds/raw/master/2024/homeworks/hw08_boosting_clustering/ds_salaries.csv
df = pd.read_csv("ds_salaries.csv")
df.head()

--2024-06-13 07:50:20--  https://github.com/hse-ds/iad-intro-ds/raw/master/2024/homeworks/hw08_boosting_clustering/ds_salaries.csv
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hse-ds/iad-intro-ds/master/2024/homeworks/hw08_boosting_clustering/ds_salaries.csv [following]
--2024-06-13 07:50:20--  https://raw.githubusercontent.com/hse-ds/iad-intro-ds/master/2024/homeworks/hw08_boosting_clustering/ds_salaries.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 210076 (205K) [text/plain]
Saving to: ‘ds_salaries.csv’

ds_salaries.csv     100%[===================>] 205.15K  --.-KB/s    in 0.03s   



,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла) Подготовка



*   Разделите выборку на train, val, test (80%, 10%, 10%)
*   Выдерите salary_in_usd в качестве таргета
*   Найдите и удалите признак, из-за которого возможен лик в данных


In [2]:
from sklearn.model_selection import train_test_split

# Выделяем таргет.
y = df['salary_in_usd']
X = df.drop('salary_in_usd', axis=1)

# Признаком, который может вызвать лик в данных является salary.
# Если мы пытаемся предсказать зарплату в долларах data scientist-а, то вряд ли мы знаем его зарплату (пусть и возможно в другой валюте).
X = X.drop('salary', axis=1)

# Разделим выборку на обучающую и тестовую.
X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.2, random_state=7)
X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=0.5, random_state=7)

## Задание 2 (0.5 балла) Линейная модель


*   Закодируйте категориальные  признаки с помощью OneHotEncoder
*   Обучите модель линейной регрессии
*   Оцените  качество через MAPE и RMSE


In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from math import sqrt

# Сохраним исходные данные до OHE.
X_train_original = X_train
X_val_original = X_val
X_test_original = X_test

# Выделим категориальные признаки.
categorical_features = X_train.columns[X_train.dtypes == "object"].values

# Выделяем явно желаемые столбцы после OHE для устранения утечки данных.
unique_values = {i: X_train[i].unique() for i in categorical_features}
unique_values = [i for i in unique_values.values()]

# OHE.
encoder = OneHotEncoder(categories=unique_values, handle_unknown='ignore')

# Преобразование X_train с помощью OneHotEncoder.
tmp = encoder.fit_transform(X_train.select_dtypes(include=['object']))

encoded_X_train = pd.DataFrame(tmp.toarray(), columns=encoder.get_feature_names_out())
X_train = X_train.drop(categorical_features, axis=1).reset_index()
X_train = pd.concat([X_train, encoded_X_train], axis=1)

# Преобразование X_test с помощью OneHotEncoder.
tmp = encoder.transform(X_test.select_dtypes(include=['object']))

encoded_X_test = pd.DataFrame(tmp.toarray(), columns=encoder.get_feature_names_out())
X_test = X_test.drop(categorical_features, axis=1).reset_index()
X_test = pd.concat([X_test, encoded_X_test], axis=1)

# Преобразование X_val с помощью OneHotEncoder.
tmp = encoder.transform(X_val.select_dtypes(include=['object']))

encoded_X_val = pd.DataFrame(tmp.toarray(), columns=encoder.get_feature_names_out())
X_val = X_val.drop(categorical_features, axis=1).reset_index()
X_val = pd.concat([X_val, encoded_X_val], axis=1)

# Обучим линейную регрессию.
linear_regression = LinearRegression().fit(X_train, y_train)

# Оценим качество через MAPE и RMSE.
print('MAPE: ', mean_absolute_percentage_error(linear_regression.predict(X_test), y_test))
print('RMSE: ', sqrt(mean_squared_error(linear_regression.predict(X_test), y_test)))

MAPE:  0.3023330202117983
RMSE:  615794218626.4669


## Задание 3 (0.5 балла) XGboost

Начнем с библиотеки xgboost.

Обучите модель `XGBRegressor` на тех же данных, что линейную модель, подобрав оптимальные гиперпараметры (`max_depth, learning_rate, n_estimators, gamma`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [4]:
from xgboost.sklearn import XGBRegressor
from sklearn.model_selection import GridSearchCV, PredefinedSplit
import numpy as np

# Головная боль для правильного подбора по валидационной выборке в GridSearchCV..
X_train_val = pd.concat([X_train, X_val], axis=0)
y_train_val = pd.concat([y_train, y_val], axis=0)
train_indices = np.full((X_train.shape[0],), -1, dtype=int)
val_indices = np.full((X_val.shape[0],), 0, dtype=int)
tmp = np.append(train_indices, val_indices)
ps = PredefinedSplit(tmp)
ps.get_n_splits()

# Подбираем гиперпараметры
params = {
    'learning_rate' : [0.01, 0.03, 0.05, 0.1],
    'n_estimators' :  range(20, 120, 20),
    'max_depth' : range(2, 10, 1),
    'gamma' : [x / 10 for x in range(0, 11, 1)]
}

model_xgb = XGBRegressor(seed=7)

# Создание экземпляра GridSearchCV для подбора гиперпараметров.
grid_search = GridSearchCV(estimator=model_xgb, param_grid=params, cv=ps, scoring='neg_root_mean_squared_error', n_jobs=-1)

# Подгонка модели на обучающих данных.
grid_search.fit(X_train_val, y_train_val)

# Вывод наилучших параметров.
print("Наилучшие параметры:", grid_search.best_params_)

Наилучшие параметры: {'gamma': 0.0, 'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}


In [43]:
%%timeit
# Обучим XGB регрессию.
XGBRegressor(seed=7, learning_rate=0.1, n_estimators=100, max_depth=4, gamma=0).fit(X_train, y_train) # Время выполнения.

494 ms ± 201 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
xgb_regression = XGBRegressor(seed=7, learning_rate=0.1, n_estimators=100, max_depth=4, gamma=0).fit(X_train, y_train)

In [45]:
%%timeit
# Оценим качество через MAPE.
mean_absolute_percentage_error(xgb_regression.predict(X_test), y_test) # Время выполнения.

32.5 ms ± 1.11 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [46]:
%%timeit
# Оценим качество через RMSE.
sqrt(mean_squared_error(xgb_regression.predict(X_test), y_test)) # Время выполнения.

33.2 ms ± 1.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [47]:
print('MAPE: ', mean_absolute_percentage_error(xgb_regression.predict(X_test), y_test)) # Время выполнения.
print('RMSE: ', sqrt(mean_squared_error(xgb_regression.predict(X_test), y_test))) # Время выполнения.

MAPE:  0.2595506174224954
RMSE:  45447.21158031558


## Задание 4 (1 балл) CatBoost

Теперь библиотека CatBoost.

Обучите модель `CatBoostRegressor`, подобрав оптимальные гиперпараметры (`depth, learning_rate, iterations`, etc.) по валидационной выборке. Оцените качество итоговой модели (MAPE, RMSE), скорость обучения и скорость предсказания.

In [14]:
from catboost import CatBoostRegressor # !pip3 install catboost, если нет catboost

params = {
    'depth' : range(2, 10, 1) ,
    'learning_rate' : [0.01, 0.05, 0.1, 0.5, 1],
    'iterations' : [10, 20, 50, 75, 100, 200] ,
}

model_cb = CatBoostRegressor(random_seed=7, verbose=False)

# Создание экземпляра GridSearchCV для подбора гиперпараметров.
grid_search = GridSearchCV(estimator=model_cb, param_grid=params, cv=ps, scoring='neg_root_mean_squared_error', n_jobs=-1)

# Подгонка модели на обучающих данных.
grid_search.fit(X_train_val, y_train_val)

# Вывод наилучших параметров.
print("Наилучшие параметры:", grid_search.best_params_)

Наилучшие параметры: {'depth': 8, 'iterations': 200, 'learning_rate': 0.1}


In [48]:
%%timeit
# Обучим CatBoost регрессию.
CatBoostRegressor(random_state=7, learning_rate=0.1, iterations=200, depth=8, verbose=False).fit(X_train, y_train) # Время выполнения.

969 ms ± 284 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
cb_regression = CatBoostRegressor(random_state=7, learning_rate=0.1, iterations=200, depth=8, verbose=False).fit(X_train, y_train)

In [50]:
%%timeit
# Оценим качество через MAPE.
mean_absolute_percentage_error(cb_regression.predict(X_test), y_test) # Время выполнения.

6.87 ms ± 71.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [51]:
%%timeit
# Оценим качество через RMSE.
sqrt(mean_squared_error(cb_regression.predict(X_test), y_test)) # Время выполнения.

6.79 ms ± 226 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [53]:
print('MAPE: ', mean_absolute_percentage_error(cb_regression.predict(X_test), y_test))
print('RMSE: ', sqrt(mean_squared_error(cb_regression.predict(X_test), y_test)))

MAPE:  0.27575814403311466
RMSE:  45974.699194474575


Для применения catboost моделей не обязательно сначала кодировать категориальные признаки, модель может кодировать их сама. Обучите catboost с подбором оптимальных гиперпараметров снова, используя pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметра cat_features. Оцените качество и время. Стало ли лучше?

In [54]:
from catboost import Pool

params = {
    'depth' : range(2, 10, 1) ,
    'learning_rate' : [0.01, 0.05, 0.1, 0.5, 1],
    'iterations' : [10, 20, 50, 75, 100, 200] ,
}

# Подбираем гиперпараметры в циклах (для передачи Pool не нашел альтернативы)
pool = Pool(data=X_train_original, label=y_train, cat_features=categorical_features)


best_depth = 0
best_learning_rate = 0
best_iterations = 0
rmse = -1

# Подбор гиперпараметров.
for depth in params['depth']:
    for learning_rate in params['learning_rate']:
        for iterations in params['iterations']:
          model_cb = CatBoostRegressor(depth=depth, learning_rate=learning_rate, iterations=iterations, random_seed=7, verbose=False)
          model_cb.fit(pool)
          y_pred = model_cb.predict(X_val_original)
          if (sqrt(mean_squared_error(y_val, y_pred)) < rmse or rmse == -1):
              rmse = sqrt(mean_squared_error(y_val, y_pred))
              best_depth = depth
              best_learning_rate = learning_rate
              best_iterations = iterations

best_params = {'depth': best_depth, 'iterations': best_iterations, 'learning_rate': best_learning_rate}

# Вывод наилучших параметров.
print("Наилучшие параметры:", best_params)

Наилучшие параметры: {'depth': 7, 'iterations': 200, 'learning_rate': 0.1}


In [55]:
%%timeit
# Обучим CatBoost регрессию.
CatBoostRegressor(random_state=7, learning_rate=0.1, iterations=200, depth=7, verbose=False).fit(pool) # Время выполнения.

2.27 s ± 824 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [59]:
cb_regression = CatBoostRegressor(random_state=7, learning_rate=0.1, iterations=200, depth=7, verbose=False).fit(pool)

In [57]:
%%timeit
# Оценим качество через MAPE.
mean_absolute_percentage_error(cb_regression.predict(X_test_original), y_test) # Время выполнения.

2.26 ms ± 92.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [58]:
%%timeit
# Оценим качество через RMSE.
sqrt(mean_squared_error(cb_regression.predict(X_test_original), y_test)) # Время выполнения.

3.48 ms ± 975 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [60]:
print('MAPE: ', mean_absolute_percentage_error(cb_regression.predict(X_test_original), y_test))
print('RMSE: ', sqrt(mean_squared_error(cb_regression.predict(X_test_original), y_test)))

MAPE:  0.2655158144247137
RMSE:  45372.420063759164


**Ответ:** при использовании Pool для передачи данных в CatBoostRegressor качество модели на тестовой выборке улучшилось (RMSE уменьшилась на ≈602), скорость предсказания осталась такой же низкой (чуть-чуть меньше, чем было), а вот скорость обучения увеличилась в два раза (выполняется больше двух секунд). В целом, можно говорить, что качество модели улучшилось ценой скорости обучения.

## Задание 5 (0.5 балла) LightGBM

И наконец библиотека LightGBM - используйте `LGBMRegressor`, снова подберите гиперпараметры, оцените качество и скорость.


In [62]:
from lightgbm import LGBMRegressor

# Подбираем гиперпараметры
params = {
    'max_depth' : range(2, 10, 1),
    'learning_rate' : [0.01, 0.05, 0.1, 0.5, 1],
    'n_estimators' :  range(20, 220, 20)
}

model_lgb = LGBMRegressor(num_leaves=31, random_state=7, verbose=-1)

# Создание экземпляра GridSearchCV для подбора гиперпараметров.
grid_search = GridSearchCV(estimator=model_lgb, param_grid=params, cv=ps, scoring='neg_root_mean_squared_error', n_jobs=-1)

# Подгонка модели на обучающих данных.
grid_search.fit(X_train_val, y_train_val)

# Вывод наилучших параметров.
print("Наилучшие параметры:", grid_search.best_params_)

Наилучшие параметры: {'learning_rate': 0.1, 'max_depth': 9, 'n_estimators': 60}


In [63]:
%%timeit
# Обучим LGBM регрессию.
LGBMRegressor(num_leaves=31, random_state=7, learning_rate=0.1, n_estimators=60, max_depth=9, verbose=-1).fit(X_train, y_train) # Время выполнения.

77.1 ms ± 8.57 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
lgb_regression = LGBMRegressor(num_leaves=31, random_state=7, learning_rate=0.1, n_estimators=60, max_depth=9, verbose=-1).fit(X_train, y_train)

In [65]:
%%timeit
# Оценим качество через MAPE.
mean_absolute_percentage_error(lgb_regression.predict(X_test), y_test) # Время выполнения.

5.13 ms ± 716 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [66]:
%%timeit
# Оценим качество через RMSE.
sqrt(mean_squared_error(lgb_regression.predict(X_test), y_test)) # Время выполнения.

4.71 ms ± 74.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [67]:
print('MAPE: ', mean_absolute_percentage_error(lgb_regression.predict(X_test), y_test))
print('RMSE: ', sqrt(mean_squared_error(lgb_regression.predict(X_test), y_test)))

MAPE:  0.2631934943136412
RMSE:  46356.16057321415


## Задание 6 (2 балла) Сравнение и выводы

Сравните модели бустинга и сделайте про них выводы, какая из моделей показала лучший/худший результат по качеству, скорости обучения и скорости предсказания? Как отличаются гиперпараметры для разных моделей?

**Ответ:** В ходе анализа качественных и временных показателей построенных моделей градиентного бустинга было выявлено, что LightGBM имеет лучшие временные показатели по скорости обучения (77ms) по сравнению с XGBoost и двумя версиями CatBoost, но немного уступает версии CatBoost c Pool по времени предсказания (хотя там речь идет буквально об одной миллисикунде, что вряд ли можно выделить в преимущество). Эта же версия CatBoost стала худшей по времени обучения (2.27s), а XgBoost - худшим по времени предсказания (33ms против ≈6ms у остальных). Лучшей по качеству моделью стал CatBoost c Pool с RMSE = 45372, а худшей - LightGBM с RMSE = 46356. Можно говорить о том, что чем выше скорость обучения, тем хуже качество модели. Что касается гиперпараметров, то все модели выбрали маленький learning_rate (0.1). Кроме того, XGBoost в качестве гиперпараметра взял маленькую глубину - 4, в то время как этот же гиперпараметр у других моделей больше 7. LightGBM взял маленькое количество деревьев (60), а XGBoost - максимальное доступное (100) (тем не менее качественный показатель у последнего лучше). CatBoost-ы, несмотря на большое выбранное количество итераций (200) показывают наилучшее качество, что говорит об устойчивости данной модели к переобучению.

# Часть 2 Кластеризация (5 баллов)

Будем работать с данными о том, каких исполнителей слушают пользователи музыкального сервиса.

Каждая строка таблицы - информация об одном пользователе. Каждый столбец - это исполнитель (The Beatles, Radiohead, etc.)

Для каждой пары (пользователь, исполнитель) в таблице стоит число - доля прослушивания этого исполнителя этим пользователем.


In [32]:
import pandas as pd
ratings = pd.read_excel("https://github.com/evgpat/edu_stepik_rec_sys/blob/main/datasets/sample_matrix.xlsx?raw=true", engine='openpyxl')
ratings.head()

,user,the beatles,radiohead,deathcab for cutie,coldplay,modest mouse,sufjan stevens,dylan. bob,red hot clili peppers,pink fluid,...,municipal waste,townes van zandt,curtis mayfield,jewel,lamb,michal w. smith,群星,agalloch,meshuggah,yellowcard
0,0,NaN,0.020417,NaN,NaN,NaN,NaN,NaN,0.030496,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,0.184962,0.024561,NaN,NaN,0.136341,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,0.028635,NaN,NaN,NaN,0.024559,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.043529,0.086281,0.034590,0.016712,0.015935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Будем строить кластеризацию исполнителей: если двух исполнителей слушало много людей примерно одинаковую долю своего времени (то есть векторы близки в пространстве), то, возможно исполнители похожи. Эта информация может быть полезна при построении рекомендательных систем.

## Задание 1 (0.5 балла) Подготовка

Транспонируем матрицу ratings, чтобы по строкам стояли исполнители.

In [33]:
# -- YOUR CODE HERE --

Выкиньте строку под названием `user`.

In [34]:
# -- YOUR CODE HERE --

В таблице много пропусков, так как пользователи слушают не всех-всех исполнителей, чья музыка представлена в сервисе, а некоторое подмножество (обычно около 30 исполнителей)


Доля исполнителя в музыке, прослушанной  пользователем, равна 0, если пользователь никогда не слушал музыку данного музыканта, поэтому заполните пропуски нулями.



In [35]:
# -- YOUR CODE HERE --
ratings.sample()

,user,the beatles,radiohead,deathcab for cutie,coldplay,modest mouse,sufjan stevens,dylan. bob,red hot clili peppers,pink fluid,...,municipal waste,townes van zandt,curtis mayfield,jewel,lamb,michal w. smith,群星,agalloch,meshuggah,yellowcard
433,433,0.024531,0.027371,0.072818,0.036925,NaN,0.031331,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Задание 2 (0.5 балла) Первая кластеризация

Примените KMeans с 5ю кластерами, сохраните полученные лейблы

In [36]:
from sklearn.cluster import KMeans

# -- YOUR CODE HERE --

Выведите размеры кластеров. Полезной ли получилась кластеризация? Почему KMeans может выдать такой результат?

In [37]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 3 (0.5 балла) Объяснение результатов

При кластеризации получилось $\geq 1$ кластера размера 1. Выведите исполнителей, которые составляют такие кластеры. Среди них должна быть группа The Beatles.

In [38]:
# -- YOUR CODE HERE --

Изучите данные, почему именно The Beatles выделяется?

Подсказка: посмотрите на долю пользователей, которые слушают каждого исполнителя, среднюю долю прослушивания.

In [39]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 4 (0.5 балла) Улучшение кластеризации

Попытаемся избавиться от этой проблемы: нормализуйте данные при помощи `normalize`.

In [40]:
from sklearn.preprocessing import normalize

# -- YOUR CODE HERE --

Примените KMeans с 5ю кластерами на преобразованной матрице, посмотрите на их размеры. Стало ли лучше? Может ли кластеризация быть полезной теперь?

In [41]:
# -- YOUR CODE HERE --

**Ответ** # -- YOUR ANSWER HERE --

## Задание 5 (1 балл) Центроиды

Выведите для каждого кластера названия топ-10 исполнителей, ближайших к центроиду по косинусной мере. Проинтерпретируйте результат. Что можно сказать о смысле кластеров?

In [ ]:
from scipy.spatial.distance import cosine


centroids = km.cluster_centers_

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

## Задание 6 (1 балл) Визуализация

Хотелось бы как-то визуализировать полученную кластеризацию. Постройте точечные графики `plt.scatter` для нескольких пар признаков исполнителей, покрасив точки в цвета кластеров. Почему визуализации получились такими? Хорошо ли они отражают разделение на кластеры? Почему?

In [ ]:
import matplotlib.pyplot as plt

# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Для визуализации данных высокой размерности существует метод t-SNE (стохастическое вложение соседей с t-распределением). Данный метод является нелинейным методом снижения размерности: каждый объект высокой размерности будет моделироваться объектов более низкой (например, 2) размерности таким образом, чтобы похожие объекты моделировались близкими, непохожие - далекими с большой вероятностью.

Примените `TSNE` из библиотеки `sklearn` и визуализируйте полученные объекты, покрасив их в цвета их кластеров

In [ ]:
from sklearn.manifold import TSNE

# -- YOUR CODE HERE --

## Задание 7 (1 балл) Подбор гиперпараметров

Подберите оптимальное количество кластеров (максимум 100 кластеров) с использованием индекса Силуэта. Зафиксируйте `random_state=42`

In [ ]:
from sklearn.metrics import silhouette_score

# -- YOUR CODE HERE --

Выведите исполнителей, ближайших с центроидам (аналогично заданию 5). Как соотносятся результаты? Остался ли смысл кластеров прежним? Расскажите про смысл 1-2 интересных кластеров, если он изменился и кластеров слишком много, чтобы рассказать про все.

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --

Сделайте t-SNE визуализацию полученной кластеризации.

In [ ]:
# -- YOUR CODE HERE --

Если кластеров получилось слишком много и визуально цвета плохо отличаются, покрасьте только какой-нибудь интересный кластер из задания выше (`c = (labels == i)`). Хорошо ли этот кластер отражается в визуализации?

In [ ]:
# -- YOUR CODE HERE --

**Ответ:** # -- YOUR ANSWER HERE --